In [1]:
import re

def extract_video_id(url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
    match = re.search(pattern, url)
    return match.group(1) if match else None


In [2]:
video_id = extract_video_id("https://youtu.be/noecHp7uZrY?si=GB3pFXmzbPGnbbE5")

print("ID is : " , video_id)

ID is :  noecHp7uZrY


In [3]:
!.venv\Scripts\activate


In [4]:



from youtube_transcript_api import YouTubeTranscriptApi


def get_transcript(video_id: str):
    


        api = YouTubeTranscriptApi()


        transcript_data = api.fetch(video_id=video_id)


        full_text = " ".join(item.text for item in transcript_data)
        
        
        
        return full_text

        


In [5]:
transcript = get_transcript(video_id)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [27]:
text_chunks = [chunk.page_content for chunk in chunks]

In [30]:
len(text_chunks)

3

In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")  # Free embedding model

  # embedding vector size: 384


In [21]:
from langchain_community.vectorstores import FAISS

In [32]:
embeddings = model.encode(text_chunks , convert_to_numpy=True)


In [39]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings



# Offline 22MB model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# chunks = list of LangChain Document objects
vector_store = FAISS.from_documents(chunks, embeddings)


C:\Users\Nirav Rupapara\AppData\Local\Temp\ipykernel_26524\562646358.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [40]:
vector_store.index_to_docstore_id


{0: 'd72f6d92-1603-4b98-8ca3-8e159b5ac1f6',
 1: '1f3f0ae9-1327-47cf-94dc-2ab54d000619',
 2: '8a810ebc-8e90-48b2-8fb7-5ed85dcb188f'}

In [47]:
retriever = vector_store.as_retriever(
    search_type="similarity",  # or "mmr"
    search_kwargs={"k": 2}     # top 3 documents
)


In [48]:
query = "what is this video about?"
docs = retriever.invoke(query)

In [49]:
docs

[Document(id='8a810ebc-8e90-48b2-8fb7-5ed85dcb188f', metadata={}, page_content="me check wait let me check okay okay uh i have to make the open video false because if i not make it false it always open in my browser and we don't need it we just need the url of that video so if we make the open video false the video will not open in our browser and we get the url of that video and let's run again the program and this time it will work for example x if i enter the name get the url of that video and for checking that you can copy the url so okay follow link sorry for that uh you can open in your browser and it's that video we are going to play so that's it for today guys i hope you liked it i hope it helped you and if there is any uh query you can ask in comment section and if there is issue while running the program in your computer or in your laptop you can ask in the comment section i will show reply your queries and that's it for today guys bye bye take care"),
 Document(id='1f3f0ae9-